In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D,Flatten, MaxPool2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input

In [ ]:
!unzip ../input/dogs-vs-cats/train.zip

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!mkdir images

In [ ]:
c=0
for i in os.listdir('./train'):
    c=c+1
    label=i.split(".")[0]
    im=cv2.imread(os.path.join('./train',i))
    img_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    if label not in os.listdir('./images'):
        os.makedirs(os.path.join('./images',label))
    cv2.imwrite(os.path.join('./images',label,i),img_rgb)
print(c)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for i in os.listdir("/kaggle/working/images/cat"):
    im=cv2.imread(os.path.join("/kaggle/working/images/cat",i))
    img_rgb=cv2.cvtColor(cv2.COLOR_BRG2RGB)
    plt.imshow(im)
    break

In [ ]:
!ls

In [ ]:
datagen= ImageDataGenerator(rotation_range=10,
                            fill_mode='nearest',
                            brightness_range=[0.5,1.5]
                           zoom_range=0.2,
                           height_shift_range=0.1,
                           width_shift_range=0.1,
                           horizontal_flip=True,
                           validation_split=0.2,
                           rescale=1/255)

In [ ]:
train_data=datagen.flow_from_directory('/kaggle/working/images',target_size=(112,112),class_mode='binary',batch_size=32,subset="training",color_mode='rgb')

In [ ]:
val_data=datagen.flow_from_directory('/kaggle/working/images',target_size=(112,112),class_mode='binary',batch_size=32,subset="validation",color_mode='rgb')

In [ ]:
train_data.next()[0].shape

In [ ]:
import matplotlib.pyplot as plt
x= train_data.next()[0]
for i in range(0,4):
    image = x[i]
    plt.imshow(image)
    plt.show()

In [ ]:
vgg=VGG16(include_top=False,weights='imagenet')

In [ ]:
for i in vgg.layers[:-5]:
    i.trainable=False

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(112,112,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(1e-5),metrics=['accuracy'])

In [ ]:
history=model.fit(train_data,validation_data=val_data,epochs=50)

In [ ]:
model.save('dogsvscats.h5')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
!pwd

In [ ]:
model=load_model('../input/dogsvscat/dogsvscats.h5')

In [ ]:
!unzip ../input/dogs-vs-cats/test1.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
res=[]
c_dog=0
c_cat=0
for i in os.listdir('./test1'):
    img=tf.keras.preprocessing.image.load_img(os.path.join('./test1',i),target_size=(112,112,3))
    np_img=tf.keras.preprocessing.image.img_to_array(img)
    np_img=np_img/255
    plt.imshow(np_img)
    np_img=np.expand_dims(np_img,axis=0)
    print(np_img.shape)
    pred=model.predict(np_img)
#     print(pred[0][0]>0.5)
    if pred[0][0]>0.5:
        c_dog=c_dog+1
        res.append('dog')
    else:
        c_cat=c_cat+1
        res.append('cat')
    

In [ ]:
c_dog,c_cat

In [ ]:
res=[0 if x=='cat' else 1 for x in res]

In [ ]:
import pandas as pd
submission=pd.read_csv('../input/dogs-vs-cats/sampleSubmission.csv')

In [ ]:
submission['label']=res

In [ ]:
submission.head()

In [ ]:
submission.to_csv('dogs_cats.csv')